In [41]:
import numpy as np
import torch
import torch.nn as nn

In [40]:
class MyConv3D:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        # Инициализация параметров свертки
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
        # Инициализация весов и смещений случайными значениями
        self.weights = np.random.randn(out_channels, in_channels, kernel_size, kernel_size, kernel_size)
        self.bias = np.random.randn(out_channels)

    def forward(self, x):
        batch_size, in_channels, in_depth, in_height, in_width = x.shape # Получаем размеры входного тензора (NCDHW)
        
        # Рассчитываем размеры выходного тензора
        out_depth = (in_depth - self.kernel_size + 2 * self.padding) // self.stride + 1
        out_height = (in_height - self.kernel_size + 2 * self.padding) // self.stride + 1
        out_width = (in_width - self.kernel_size + 2 * self.padding) // self.stride + 1
        
        # Добавляем отступы к входным данным
        out = np.zeros((batch_size, self.out_channels, out_depth, out_height, out_width))

        # массив для выходных данных
        padded_input = np.pad(x, ((0, 0), (0, 0), (self.padding, self.padding), (self.padding, self.padding), (self.padding, self.padding)), mode='constant')

        # Выполняем свертку
        for d in range(out_depth):
            for h in range(out_height):
                for w in range(out_width):
                    receptive_field = padded_input[:, :, d * self.stride : d * self.stride + self.kernel_size,
                                                   h * self.stride : h * self.stride + self.kernel_size,
                                                   w * self.stride : w * self.stride + self.kernel_size]
                    # Используем tensordot для свертки и добавляем смещение
                    out[:, :, d, h, w] = np.tensordot(receptive_field, self.weights, axes=([1, 2, 3, 4], [1, 2, 3, 4])) + self.bias

        return out

In [31]:
def test_conv3d(in_channels, out_channels, kernel_size, stride, padding):
    # Генерация случайных входных данных
    batch_size = 1
    in_depth, in_height, in_width = 64, 64, 64
    input_data = np.random.randn(batch_size, in_channels, in_depth, in_height, in_width)

    # Инициализация MyConv3D
    my_conv3d = MyConv3D(in_channels, out_channels, kernel_size, stride, padding)
    my_output = my_conv3d.forward(input_data)

    # Инициализация PyTorch Conv3d
    torch_conv3d = nn.Conv3d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
    torch_conv3d.weight.data = torch.from_numpy(my_conv3d.weights)
    torch_conv3d.bias.data = torch.from_numpy(my_conv3d.bias)
    torch_output = torch_conv3d(torch.tensor(input_data))

    # Сравнение выходных данных
    assert np.allclose(my_output, torch_output.detach().numpy()), "Тест провален!"
    
    print("Тест пройден успешно!")

In [43]:
# Проверка базового случая
test_conv3d(3, 10, 3, 1, 1)

Тест пройден успешно!


In [44]:
# Проверка при другом размере ядра
test_conv3d(2, 5, 5, 1, 1)

Тест пройден успешно!


In [45]:
# Проверка на большем отступе
test_conv3d(3, 1, 3, 2, 2)

Тест пройден успешно!


In [46]:
# Проверка при большем количестве каналов
test_conv3d(4, 8, 3, 1, 0)

Тест пройден успешно!
